In [50]:
%run packages.py

In [51]:
from data_import_enginnering import DataImportEnginnering
from metric_selection import Pair_Selection
from data_visualization import Data_Visualization
import importlib

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Importation of data into a dictionary

In [52]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
tables = pd.read_html(url)
Stock_list = pd.DataFrame(tables[0])
Stock_list.drop(['Security','GICS Sub-Industry','Headquarters Location','CIK','Founded'],axis = 1, inplace=True)
Stock_list['Date added'] = pd.to_datetime(Stock_list['Date added'])
Stock_list = Stock_list[( Stock_list['Date added'] < '2019-12-31') & (Stock_list['Date added']>'2015-01-01')][['Symbol','GICS Sector']]


In [53]:
stock_list = list(Stock_list['Symbol'].unique())
Stock_list_sector = Stock_list.groupby('GICS Sector')['Symbol'].unique().to_dict()
sector_list = list(Stock_list_sector.keys())
pairs = {k:[tuple(sorted((i,j))) for i in Stock_list_sector[k] for j in Stock_list_sector[k] if j!=i] for k in sector_list}
pairs = {k :sorted(list(set(pairs[k]))) for k in sector_list}

In [54]:
start_date = '2018-01-01'
end_date = '2019-12-31'
period = '1d'
colone = ['Close']


In [55]:
Enginner = DataImportEnginnering(stock_list,start_date=start_date,end_date=end_date,period=period,colone=colone)

In [56]:
raw_data = Enginner.RawData()
raw_data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

{'AOS': Price           Close
 Ticker            AOS
 Date                 
 2018-01-02  54.426807
 2018-01-03  54.754105
 2018-01-04  55.010609
 2018-01-05  55.718250
 2018-01-08  55.983620
 ...               ...
 2019-12-23  43.067318
 2019-12-24  43.396217
 2019-12-26  43.615494
 2019-12-27  43.679443
 2019-12-30  43.460175
 
 [502 rows x 1 columns],
 'AMD': Price           Close
 Ticker            AMD
 Date                 
 2018-01-02  10.980000
 2018-01-03  11.550000
 2018-01-04  12.120000
 2018-01-05  11.880000
 2018-01-08  12.280000
 ...               ...
 2019-12-23  45.459999
 2019-12-24  46.540001
 2019-12-26  46.630001
 2019-12-27  46.180000
 2019-12-30  45.520000
 
 [502 rows x 1 columns],
 'ALB': Price            Close
 Ticker             ALB
 Date                  
 2018-01-02  120.240555
 2018-01-03  120.478226
 2018-01-04  119.042992
 2018-01-05  120.414253
 2018-01-08  125.652428
 ...                ...
 2019-12-23   68.524300
 2019-12-24   68.420227
 2019-12-26   68.

# 2. Normalisation of data

In [57]:
raw_data_normalized = {}
for key, value in raw_data.items():
    raw_data_normalized[key] = Enginner.normalize(value)
raw_dataframe = Enginner.Dictionnary_to_Dataframe(raw_data)

/Users/julesarzel/Desktop/Pairs trading/Pair_trading-1/data_import_enginnering.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.data_normalized.fillna(method = 'ffill',inplace = True)
/Users/julesarzel/Desktop/Pairs trading/Pair_trading-1/data_import_enginnering.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.data_normalized.fillna(method = 'ffill',inplace = True)
/Users/julesarzel/Desktop/Pairs trading/Pair_trading-1/data_import_enginnering.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.data_normalized.fillna(method = 'ffill',inplace = True)
/Users/julesarzel/Desktop/Pairs trading/Pair_trading-1/data_import_enginnering.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated

In [82]:
dataframe_normalized = Enginner.Dictionnary_to_Dataframe(raw_data_normalized)

In [83]:
dataframe_normalized= dataframe_normalized.stack().stack().droplevel(level =2).unstack()
raw_dataframe= raw_dataframe.stack().stack().droplevel(level =2).unstack()

/var/folders/3_/t8l3yh8s28z9_jbq7gqy7byr0000gn/T/ipykernel_3178/628406389.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  dataframe_normalized= dataframe_normalized.stack().stack().droplevel(level =2).unstack()
/var/folders/3_/t8l3yh8s28z9_jbq7gqy7byr0000gn/T/ipykernel_3178/628406389.py:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  raw_dataframe= raw_dataframe.stack().stack().droplevel(level =2).unstack()


# 3. Selection of pairs (2) and new dictionary for them

In [84]:
Metric = Pair_Selection(dataframe_normalized,number_of_pair=2,stock_list_sector=Stock_list_sector,sector_list=sector_list)

In [85]:
distance = Metric.compute_euclidian_distance()


In [86]:

selected_pairs = Metric.paire_selection(distance,number_of_pair=2,stock_list_sector=Stock_list_sector,sector_list=sector_list,pairs=pairs)

In [87]:
data1 = dataframe_normalized.copy()
data2 = raw_dataframe.copy()
# The goal would be to stock every small data frame in a dictionnay to help doing the steps for the reste 
dataframes_dictionnary_norm = {}
dataframes_dictionnary_raw ={}
for sector in sector_list: 
    for value in selected_pairs[sector]:
        
        dataframes_dictionnary_norm[value] = data1[list(value)]
        dataframes_dictionnary_raw[value] = data2[list(value)]

In [88]:
from Spread import Spread


In [89]:
compute_spread = Spread(dataframes_dictionnary_norm)
x = compute_spread.dollar_neutral_spread()

/Users/julesarzel/Desktop/Pairs trading/Pair_trading-1/Spread.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values['Delta'] = np.abs(values[key[0]]/values[key[1]])
/Users/julesarzel/Desktop/Pairs trading/Pair_trading-1/Spread.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values['Delta_norm'] = (values['Delta'] - values['Delta'].rolling(10).mean()) / values['Delta'].rolling(10).std()
/Users/julesarzel/Desktop/Pairs trading/Pair_trading-1/Spread.py:16: SettingWithCopyWarning: 
A value is trying 

In [90]:
x

{('FOX',
  'FOXA'): Ticker           FOX      FOXA  Delta_norm
 Date                                      
 2018-01-02       NaN       NaN         NaN
 2018-01-03       NaN       NaN         NaN
 2018-01-04       NaN       NaN         NaN
 2018-01-05       NaN       NaN         NaN
 2018-01-08       NaN       NaN         NaN
 ...              ...       ...         ...
 2019-12-23  0.631740  0.666870    2.160849
 2019-12-24  0.619287  0.652902    1.850165
 2019-12-26  0.648025  0.677233    1.841222
 2019-12-27  0.640841  0.662815    1.788614
 2019-12-30  0.613539  0.633076    1.470355
 
 [502 rows x 3 columns],
 ('LYV',
  'TMUS'): Ticker           LYV      TMUS  Delta_norm
 Date                                      
 2018-01-02  0.163553  0.295082         NaN
 2018-01-03  0.159762  0.280084         NaN
 2018-01-04  0.153263  0.258807         NaN
 2018-01-05  0.181695  0.314615         NaN
 2018-01-08  0.174384  0.332055         NaN
 ...              ...       ...         ...
 2019-12-23

In [91]:
for key,values in dataframes_dictionnary_norm.items():
    values['Delta'] = np.abs(values[key[0]] - values[key[1]])
    values['Delta_norm'] = (values['Delta'] - values['Delta'].rolling(10).mean()) / values['Delta'].rolling(10).std()
    values.drop(['Delta'], axis = 1, inplace=True )

dataframes_dictionnary_norm


/var/folders/3_/t8l3yh8s28z9_jbq7gqy7byr0000gn/T/ipykernel_3178/1693056066.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values['Delta'] = np.abs(values[key[0]] - values[key[1]])
/var/folders/3_/t8l3yh8s28z9_jbq7gqy7byr0000gn/T/ipykernel_3178/1693056066.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values['Delta_norm'] = (values['Delta'] - values['Delta'].rolling(10).mean()) / values['Delta'].rolling(10).std()
/var/folders/3_/t8l3yh8s28z9_jbq7gqy7byr0000gn/T/ipykernel_3178/1693056066.py:4: Setti

{('FOX',
  'FOXA'): Ticker           FOX      FOXA  Delta_norm
 Date                                      
 2018-01-02       NaN       NaN         NaN
 2018-01-03       NaN       NaN         NaN
 2018-01-04       NaN       NaN         NaN
 2018-01-05       NaN       NaN         NaN
 2018-01-08       NaN       NaN         NaN
 ...              ...       ...         ...
 2019-12-23  0.631740  0.666870   -2.611076
 2019-12-24  0.619287  0.652902   -1.969457
 2019-12-26  0.648025  0.677233   -1.859171
 2019-12-27  0.640841  0.662815   -1.892731
 2019-12-30  0.613539  0.633076   -1.585892
 
 [502 rows x 3 columns],
 ('LYV',
  'TMUS'): Ticker           LYV      TMUS  Delta_norm
 Date                                      
 2018-01-02  0.163553  0.295082         NaN
 2018-01-03  0.159762  0.280084         NaN
 2018-01-04  0.153263  0.258807         NaN
 2018-01-05  0.181695  0.314615         NaN
 2018-01-08  0.174384  0.332055         NaN
 ...              ...       ...         ...
 2019-12-23

In [96]:
beta_neutral = compute_spread.beta_neutral_spread()
beta_neutral

/Users/julesarzel/Desktop/Pairs trading/Pair_trading-1/Spread.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values['Delta'] = np.abs(values[key[0]] - values[key[1]]*beta)
/Users/julesarzel/Desktop/Pairs trading/Pair_trading-1/Spread.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values['Delta_norm'] = (values['Delta'] - values['Delta'].rolling(10).mean()) / values['Delta'].rolling(10).std()
/Users/julesarzel/Desktop/Pairs trading/Pair_trading-1/Spread.py:38: SettingWithCopyWarning: 
A value is 

{('FOX',
  'FOXA'): Ticker           FOX      FOXA  Delta_norm
 Date                                      
 2018-01-02       NaN       NaN         NaN
 2018-01-03       NaN       NaN         NaN
 2018-01-04       NaN       NaN         NaN
 2018-01-05       NaN       NaN         NaN
 2018-01-08       NaN       NaN         NaN
 ...              ...       ...         ...
 2019-12-23  0.631740  0.666870         NaN
 2019-12-24  0.619287  0.652902         NaN
 2019-12-26  0.648025  0.677233         NaN
 2019-12-27  0.640841  0.662815         NaN
 2019-12-30  0.613539  0.633076         NaN
 
 [502 rows x 3 columns],
 ('LYV',
  'TMUS'): Ticker           LYV      TMUS  Delta_norm
 Date                                      
 2018-01-02  0.163553  0.295082         NaN
 2018-01-03  0.159762  0.280084         NaN
 2018-01-04  0.153263  0.258807         NaN
 2018-01-05  0.181695  0.314615         NaN
 2018-01-08  0.174384  0.332055         NaN
 ...              ...       ...         ...
 2019-12-23

In [95]:
compute_spread2 = Spread(dataframes_dictionnary_raw)
dollar_neutral = compute_spread2.dollar_neutral_spread2()
dollar_neutral

/Users/julesarzel/Desktop/Pairs trading/Pair_trading-1/Spread.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/julesarzel/Desktop/Pairs trading/Pair_trading-1/Spread.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/julesarzel/Desktop/Pairs trading/Pair_trading-1/Spread.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

{('FOX',
  'FOXA'): Ticker            FOX       FOXA  Delta_norm
 Date                                        
 2018-01-02        NaN        NaN         NaN
 2018-01-03        NaN        NaN         NaN
 2018-01-04        NaN        NaN         NaN
 2018-01-05        NaN        NaN         NaN
 2018-01-08        NaN        NaN         NaN
 ...               ...        ...         ...
 2019-12-23  33.958374  34.885113         NaN
 2019-12-24  33.837849  34.740753         NaN
 2019-12-26  34.115974  34.992222         NaN
 2019-12-27  34.046448  34.843201         NaN
 2019-12-30  33.782223  34.535847         NaN
 
 [502 rows x 3 columns],
 ('LYV',
  'TMUS'): Ticker            LYV       TMUS  Delta_norm
 Date                                        
 2018-01-02  42.650002  63.061272         NaN
 2018-01-03  42.509998  62.637844         NaN
 2018-01-04  42.270000  62.037170         NaN
 2018-01-05  43.320000  63.612717         NaN
 2018-01-08  43.049999  64.105080         NaN
 ...           